In [1]:
import tensorflow as tf
import numpy as np
from preposessing_data import generate_batch, data_generator

In [2]:
ENCODER_INPUT_SIZE = 4096
HIDDEN_LAYER_SIZE = 1024
EMBEDDING_SIZE = 1024
BOS = 0
EOS = 1
BATCH_SIZE = 32

In [3]:
X, y_inputs, y_targets, word_idx, idx_word, num_of_words, max_length, sequence_lengths = data_generator('./data/training_data', './data/training_label.json', 2)

starting load data......
encoding data........
adding special symbol......
[[  0   4   5 ...   3   3   3]
 [  0   4  10 ...   3   3   3]
 [  0   4  15 ...   3   3   3]
 ...
 [  0   4   5 ...   3   3   3]
 [  0   8 397 ...   3   3   3]
 [  0   4  10 ...   3   3   3]] [[  4   5   6 ...   3   3   3]
 [  4  10  11 ...   3   3   3]
 [  4  15  16 ...   3   3   3]
 ...
 [  4   5 111 ...   3   3   3]
 [  8 397 507 ...   3   3   3]
 [  4  10  17 ...   3   3   3]]
Done data generation!


In [4]:
sess = tf.InteractiveSession()
encoder_inputs = tf.placeholder(tf.float32, shape = [None, None, ENCODER_INPUT_SIZE])
decoder_inputs = tf.placeholder(tf.int32, shape = [None, None])
decoder_targets = tf.placeholder(tf.int32, shape = [None, None])
sequence_length = tf.placeholder(tf.int32, shape = [None])
sequence_length_fake = tf.placeholder(tf.int32, shape = [None])

In [5]:
encoder_cell = tf.contrib.rnn.LSTMCell(HIDDEN_LAYER_SIZE)
encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell, 
                                       encoder_inputs, 
                                       dtype=tf.float32)

In [6]:
decoder_cell = tf.contrib.rnn.LSTMCell(HIDDEN_LAYER_SIZE)
output_layer = tf.layers.Dense(num_of_words, activation=tf.nn.softmax)
# embedding for decoder
embeddings = tf.Variable(tf.random_uniform([num_of_words, EMBEDDING_SIZE], -1.0, 1.0), dtype=tf.float32)
decoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, decoder_inputs)

training_helper = tf.contrib.seq2seq.TrainingHelper(decoder_inputs_embedded, 
                                           sequence_length_fake)
training_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, 
                                          training_helper, 
                                          encoder_state,
                                          output_layer)
# unrolling the decoder layer
training_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder, 
                                                  impute_finished = True, maximum_iterations = max_length)






In [7]:
inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings, 
                                                      tf.fill([BATCH_SIZE], BOS), 
                                                      EOS)

inference_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, 
                                              inference_helper, 
                                              encoder_state, 
                                              output_layer)

inference_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder, 
                                                      impute_finished=True, maximum_iterations = max_length)




In [8]:
training_logits = tf.identity(training_outputs.rnn_output, name='logits')
masks = tf.sequence_mask(sequence_length, max_length, name = 'mask', dtype=tf.float32)

In [9]:
training_logits
print(max_length)

41


In [10]:
# Define training
with tf.name_scope("optimization"):
    # Loss function - weighted softmax cross entropy
    cost = tf.contrib.seq2seq.sequence_loss(
        training_logits,
        decoder_targets,
        masks)

    # Optimizer
    optimizer = tf.train.AdamOptimizer()

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

In [11]:
# training start

sess.run(tf.global_variables_initializer())

epoc = 20
fake_max_sequence = np.array([max_length] * BATCH_SIZE)
for i in range(epoc):

    for j in range(len(X) // BATCH_SIZE):
        X_batch, y_inputs_batch, y_targets_batch, sequence_length_batch = generate_batch(X, y_inputs, y_targets, word_idx,
         sequence_lengths, BATCH_SIZE)
        print(y_inputs_batch.shape, y_targets_batch.shape, max(sequence_length_batch))
        _, loss= sess.run([train_op, cost], feed_dict= {encoder_inputs : X_batch, decoder_inputs : y_inputs_batch,
                                              decoder_targets: y_targets_batch, sequence_length : sequence_length_batch,
                                                                      sequence_length_fake : fake_max_sequence})
        print("loss : {0}".format(loss))

[ 7  8 18 ... 16  8 11]
(32, 41) (32, 41) 23
loss : 6.436155319213867
[ 7  8 18 ... 16  8 11]
(32, 41) (32, 41) 25
loss : 6.393674373626709
[ 7  8 18 ... 16  8 11]
(32, 41) (32, 41) 25
loss : 6.354521751403809
[ 7  8 18 ... 16  8 11]
(32, 41) (32, 41) 21
loss : 6.321094989776611
[ 7  8 18 ... 16  8 11]
(32, 41) (32, 41) 24
loss : 6.294537544250488
[ 7  8 18 ... 16  8 11]
(32, 41) (32, 41) 26
loss : 6.243799686431885
[ 7  8 18 ... 16  8 11]
(32, 41) (32, 41) 30
loss : 6.264980792999268
[ 7  8 18 ... 16  8 11]
(32, 41) (32, 41) 30
loss : 6.270837306976318
[ 7  8 18 ... 16  8 11]
(32, 41) (32, 41) 26
loss : 6.247490882873535
[ 7  8 18 ... 16  8 11]
(32, 41) (32, 41) 21
loss : 6.245001792907715
[ 7  8 18 ... 16  8 11]
(32, 41) (32, 41) 40
loss : 6.2640156745910645
[ 7  8 18 ... 16  8 11]
(32, 41) (32, 41) 29


KeyboardInterrupt: 

In [ ]:
fake_max_sequence